<a href="https://www.kaggle.com/code/enricofindley/corn-leaf-disease-classification-with-fcm-lstm?scriptVersionId=140148685" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# README
#### This is my thesis project, if some of you need to use a portion or all of my code feel free to copy it but please cite my thesis <a href="http://repo.darmajaya.ac.id/11576/">here</a>

# Installing Dependencies

In [7]:
# install split folders for splitting image data into train,val, and test.
%pip install split-folders
%pip install scikit-fuzzy
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import os
import cv2
import skfuzzy as fuzz
import pandas as pd 
from time import time
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
import splitfolders
import shutil
from keras.utils import np_utils
warnings.filterwarnings('ignore')

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 19.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894088 sha256=26e5de73ed3adf3b4bd012e83263c1386fa5aea90337729b916f195e2d9e01fb
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy
Note: you may need to restart the kernel to use updated packages.


# Image Dataset Splitting

In [2]:
file_path='/kaggle/input/new-bangladeshi-crop-disease/BangladeshiCrops/BangladeshiCrops/Crop___Disease/Corn'
splitted_folder='data'
if not os.path.exists(splitted_folder):
   os.makedirs(splitted_folder)
else:
  # Deleting an non-empty folder
  shutil.rmtree(splitted_folder, ignore_errors=True)
  print("Deleted '%s' directory successfully" % splitted_folder)

  os.makedirs(splitted_folder)

def train_test_valid(images_folder=file_path,splitted_folder=splitted_folder):
  input_folder = images_folder
  output_folder = splitted_folder
  splitfolders.fixed(input_folder,output_folder, seed = 1337, fixed = (100,100), oversample=True, group_prefix = None)
train_test_valid()

Copying files: 3852 files [00:31, 124.18 files/s]
Oversampling: 4 classes [00:00, 26.21 classes/s]


# Declaring Global Variables

In [8]:
CATEGORIES = ["Corn___Common_Rust","Corn___Gray_Leaf_Spot","Corn___Healthy", "Corn___Northern_Leaf_Blight"]
IMG_SIZE = 128

TRAIN_DATADIR = "/kaggle/working/data/train"
VALIDATION_DATADIR = "/kaggle/working/data/val"
TEST_DATADIR = "/kaggle/working/data/test"

SEG_TRAIN_DATADIR = "/kaggle/working/data/segmented/train"
SEG_VALIDATION_DATADIR = "/kaggle/working/data/segmented/val"
SEG_TEST_DATADIR = "/kaggle/working/data/segmented/test"
clusters = [2]

# Declaring Global Helper Function

In [12]:
def change_color_fuzzycmeans(cluster_membership, clusters):
    img = []
    for pix in cluster_membership.T:
        img.append(clusters[np.argmax(pix)])
    return img

def segmentImage(data_path, result_path):
    labels=[]
    segmented_images=[]
    for category in CATEGORIES:  
        index=0
        path = os.path.join(data_path,category)  # create path to image
        seg_path=os.path.join(result_path,category)  # create path to image
        class_num = CATEGORIES.index(category)  # get the classification
        
        for img in os.listdir(path):  # iterate over each image 
            img_input = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
            img_resized= cv2.resize(img_input, (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)
            gray_img = img_resized.reshape(( img_resized.shape[0] *  img_resized.shape[1], 1))          
            img = np.reshape(gray_img, (IMG_SIZE,IMG_SIZE,1)).astype(np.uint8)
            shape = np.shape(img)

            for i,cluster in enumerate(clusters):
                        
                # Fuzzy C Means
                new_time = time()
                    
                cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(gray_img.T, cluster, 2, error=0.00001, maxiter=100, init=None,seed=42)  
                new_img = change_color_fuzzycmeans(u,cntr)
                    
                fuzzy_img = np.reshape(new_img,shape).astype(np.uint8)  
                ret, seg_img = cv2.threshold(fuzzy_img,np.max(fuzzy_img)-1,255,cv2.THRESH_BINARY)

                # Add the color mask to the original image using alpha blending
                alpha = 0.6
                result = cv2.addWeighted(img, alpha, seg_img, 1-alpha, 0)     

            segmented_images.append(seg_img)
            labels.append(class_num)
            index=index+1
            cv2.imwrite(os.path.join(seg_path , 'segmented'+str(index)+'.png'), result)
    return segmented_images,labels

# Segmenting All Corn Image

In [11]:
train_img, train_label = segmentImage(TRAIN_DATADIR, SEG_TRAIN_DATADIR)
val_img, val_label = segmentImage(VALIDATION_DATADIR, SEG_VALIDATION_DATADIR)
test_img, test_label = segmentImage(TEST_DATADIR, SEG_TEST_DATADIR)

Fuzzy time for cluster 2
0.3181583881378174 seconds
Fuzzy time for cluster 2
0.3339865207672119 seconds
Fuzzy time for cluster 2
0.2520790100097656 seconds
Fuzzy time for cluster 2
0.3072376251220703 seconds
Fuzzy time for cluster 2
0.4302678108215332 seconds
Fuzzy time for cluster 2
0.39544248580932617 seconds
Fuzzy time for cluster 2
0.33088016510009766 seconds
Fuzzy time for cluster 2
0.31877827644348145 seconds
Fuzzy time for cluster 2
0.43797731399536133 seconds
Fuzzy time for cluster 2
0.36650943756103516 seconds
Fuzzy time for cluster 2
0.2784430980682373 seconds
Fuzzy time for cluster 2
0.48232173919677734 seconds
Fuzzy time for cluster 2
0.3181009292602539 seconds
Fuzzy time for cluster 2
0.3999054431915283 seconds
Fuzzy time for cluster 2
0.36762547492980957 seconds
Fuzzy time for cluster 2
0.3022313117980957 seconds
Fuzzy time for cluster 2
0.35926032066345215 seconds
Fuzzy time for cluster 2
0.27251100540161133 seconds
Fuzzy time for cluster 2
0.3069005012512207 seconds
Fuz

KeyboardInterrupt: 